<a id="bql-asset-top"></a>
<div style="width: 50px; height: 50px; border-radius: 50%; background: #2E86C1; margin-bottom: 0px; text-align: center;">
    <img src="../img/Welcome Page/orbit.png" style="max-width: 150%; max-height: 150%; right: 25%; bottom: 13px; position: relative;">
</div>
<h1 style="margin-top: 20px; margin-bottom: 5px;">BQL For Asset Classes<br><span style="color: orange; margin-bottom: 0px;">Functions & Universes for Different Asset Classes</span></h1>
<h4 style="color: orange; margin-top: 0px;">(15-min Read)</h4>

By the end of this document, you will be able to:
1. Explain some of the nuances in BQL queries across different asset classes
1. Understand where to find available resources for BQL relative to a specific asset class

### Contents
- [Using BQL for Various Asset Classes](#bqlac-intro)
- [Fixed Income](#bqlac-fi)
- [Equities](#bqlac-eq)
- [ESG](#bqlac-esg)
- [Funds](#bqlac-funds)
- [Economics](#bqlac-econ)

<a href="../2. Launch/4 Launch Project.ipynb">&larr; Back to Launch: Final Project</a>&emsp; | &emsp;
<a href="2 Dashboards.ipynb">Continue to Orbit: Dashboards &rarr;</a>

In [4]:
# Setup Environment for Exampels
import bql
import pandas as pd
bq = bql.Service()

---
<a id="bqlac-intro"></a>
<h1>Using BQL for Various Asset Classes<br><span style="color: orange;">Introduction</span></h1>

BQL is an extensive topic, and the syntax of your queries can vary depending on the asset class you're analyzing. The vast majority of BQL functions, fields, and universes are outside the scope of this tutorial, and you can find more comprehensive documentation in the Example Projects section of your Project Library.

<img src="../img/Orbit/bql_directory.png">

This tutorial will discuss a few ways you can refine your BQL code to work with specific asset classes. 

[&uarr; Return to Top](#bql-asset-top)

---
<a id="bqlac-fi"></a>
<h1>BQL for Fixed Income<br><span style="color: orange;">Special Universe Handlers for FI</span></h1>

There are many BQL fields available especially for the Fixed Income world, but what distinguishes FI apart from other BQL assets are its universe handlers. There are several universes we can use when working with BQL for credit data:

* <b>bq.univ.bondsuniv()</b>: This represents all bonds which you can filter, best used with the "active" parameter to exclude all matured bonds. 

* <b>bq.univ.bonds()</b>: to pull all bonds for another universe. ie:
 *  bq.univ.bonds("VOD LN Equity")
 *  bq.univ.bonds(bq.univ.members("SPX Index")) 
 
* <b>bq.univ.loans()</b>: to pull loans
* <b>bq.univ.debt()</b>: to pull bonds, loans, preferreds

The example below uses the `bondsuniv()` handler along with the `filter()` function to find active issues with more than $100 billion outstanding.


In [7]:
# Run this cell with the Run button in the Jupyter toolbar
active_bonds = bq.univ.bondsuniv('Active')
amount_out = bq.data.amt_outstanding(Currency='USD')
filtered_bonds = active_bonds.filter(amount_out > 100*10**9)

req = bql.Request(filtered_bonds, {'Issuer': bq.data.issuer(),
                                   'Outstanding (Billions of USD)': amount_out / 10**9})
res = bq.execute(req)
df = pd.concat([field.df()[field.name] for field in res], axis=1)
df.head()

,Issuer,Outstanding (Billions of USD)
ID,,
91282CAE Govt,US TREASURY N/B,133.019
912828ZQ Govt,US TREASURY N/B,109.880
9127963W Govt,TREASURY BILL,149.150
9127964C Govt,TREASURY BILL,146.856
9127964D Govt,TREASURY BILL,148.841


For more information about Fixed Income BQL Queries, see <a href="../exampleroot/1e2e6cfc72254cdabb84868cd51d613d/0 Welcome.ipynb">Example Projects &rarr; BQL Resources &rarr; BQL for Fixed Income</a>

[&uarr; Return to Top](#bql-asset-top)

---
<a id="bqlac-eq"></a>
<h1>BQL for Equities<br><span style="color: orange;">Parameters for Fundamental Data</span></h1>

Pulling fundamental data on equity securities introduces a number of special parameters. Here are just some of the most commonly used ones. For a full detailed reference of parameters, please visit our <a href="https://blinks.bloomberg.com/screens/docs%202085646">Fundamentals Reference Guide</a>.

| Name| Parameter ID | Default Value | Enum Values | Example |
|---|---|---|---|---|
| Period Type | fpt | LTM | LTM/P/A/S/Q/BA/BS/BQ/BT  | fpt='A'; fpt='BA' |  
| Period Offset | fpo | - | integer , integer＋R＋period , integer＋F＋period | fpo='1'; fpo='-1' |
| Period Reference | fpr | - | Date (YYYY-MM-DD), Year (YYYY), Year＋Quarter (YYYYQX) | fpr='2019'; fpr='2019Q4'|
| Period Type Source | fpts | P | P/A/Q/S | fpt='BT'・fpts='A': This will calculate 12m blended forward based on next two annuals |  
| Adjustment for abnormal items | adj | N | Y/N | |
| Actuals/Estimates | ae | ae | AE/A/E | fpr='2018'・ae='E': This will return estimates for 2018 |
| Estimates Source | est_source | BST | BST/BLI/BPE/CGD | est_source='CGD' |
| Currency | currency | Reported Currency | - | Currency='JPY' |
| Corporate Actions | ca_adj | splits | full, raw, splits | ca_adj='full' |  

<br>The example below uses some BQL fields unique to equity universes to pull fundamental data for a single stock.


In [8]:
# Run this cell with the Run button in the Jupyter toolbar

# Define the universe
univ = 'AMZN US Equity'
# Define the data items. 
cash = bq.data.bs_cash_near_cash_item() / 1000000
liab = bq.data.bs_mkt_sec_other_st_invest() / 1000000
marketable_sec = bq.data.bs_mkt_sec_other_st_invest() / 10 ** 6
accounts_receivable = bq.data.bs_accts_rec_excl_notes_rec() / 10 ** 6
# You can combine variables to create custom fields
quick_ratio = (cash + marketable_sec + accounts_receivable) / liab
# Create a dictionary and assign names to data items 
items = {'Cash': cash, 
         'Marketable Securities': marketable_sec,
         'Accounts Receivable':accounts_receivable,
         'Liabilities': liab, 
         'Quick Ratio':quick_ratio}
#Request data, handle response
req = bql.Request(univ,items)
res = bq.execute(req)
data1 = pd.DataFrame({r.name : r.df()[r.name] for r in res})
data1

,Cash,Marketable Securities,Accounts Receivable,Liabilities,Quick Ratio
ID,,,,,
AMZN US Equity,29930.0,38472.0,20832.0,38472.0,2.319453


For more information about Equity BQL Queries, see <a href="../exampleroot/6f5ff44942574c4688dbfd272c512be9/0 Welcome.ipynb">Example Projects &rarr; BQL Resources &rarr; BQL for Equities</a>

[&uarr; Return to Top](#bql-asset-top)

---
<a id="bqlac-esg"></a>
<h1>BQL for ESG<br><span style="color: orange;">Special Queries for Sustainable Investing</span></h1>

Bloomberg has a wealth of data on sustainable investing that BQuant developers can access.

- Over 1,000 fields spanning E, S, G for 11,500+ Companies with over 10+ years history with Bloomberg Transparency. Full list available on <a href="https://blinks.bloomberg.com/screens/BQLX">BQLX</a>
- Bloomberg collected data from annual reports, CSR, working with organisations such as Sustainable Accounting Standards Board (SASB) and United Nations Global Compact (UNGC) to push for more companies to release higher quality and quantity of data
- Bloomberg proprietary indices such as Bloomberg ESG Data Index (BESGPro Index), Gender Equality Index (BGEI Index)
- Index collaborations with third party providers such as MSCI and SASB text

### Screening for ESG Data


Screening for ESG data is one of the most common needs for analysis. There are two main methods to look at this:

* Leverage <code>bq.univ.filter()</code> to create your own factors and apply positive/negative screening to your universe.

* Screen ESG data for availability across datasets and across time

The query below screens for companies with high positive changes in the percentage of female employees.

In [10]:
#Define start universe
index = bq.univ.members('BESGPro Index')
#Percentge change in woman employed in the last 2 years. 
women_chg = bq.data.pct_women_employees(fpt = 'A', fpo = bq.func.range('-2','0')).net_chg()
# rank universe by % change in woman employed
women_chg_rank = women_chg.grouprank()
# filter top 5
women_chg_filter = women_chg_rank <= 5
#Apply filter
filtered_index = bq.univ.filter(index, women_chg_filter)
#Send Request to service, convert response to DataFrame
req = bql.Request(filtered_index, {'Top Chg in Women Employees':women_chg})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,Top Chg in Women Employees
ID,,,,
980 HK Equity,2020-05-08,2020-11-26,2019-12-31,38.244001
ODPV3 BZ Equity,2020-03-05,2020-11-26,2019-12-31,48.070002
4152 TT Equity,2020-02-15,2020-11-26,2019-12-31,30.869995
INW IM Equity,2020-03-17,2020-11-26,2019-12-31,37.400002
UPRO RM Equity,2020-03-10,2020-11-26,2019-12-31,42.000000


For more information about ESG BQL Queries, see <a href="../exampleroot/7375305a4ccd48648475314192e49c2b/0 Welcome.ipynb">Example Projects &rarr; BQL Resources &rarr; BQL for ESG</a>

[&uarr; Return to Top](#bql-asset-top)

---
<a id="bqlac-funds"></a>
<h1>BQL for Funds<br><span style="color: orange;">Access Fund Holding Data</span></h1>

Use the `members(<Fund/Index Ticker>, type='HOLDINGS')` universe function on funds to access holding data. Using the `id()` function in BQL's `GET` statement will return a DataFrame of all holdings data.

In [11]:
# Run this cell with the Run button in the Jupyter toolbar
# set universe to fund members, set type='holdings'
fund_ticker = 'SPY US Equity'
universe = bq.univ.members(fund_ticker, type='holdings')

# return all columns in the holdings dataframe
all_holdings_data = bq.data.id()

# build and execute request
req = bql.Request(universe, all_holdings_data)
res = bq.execute(req)

# display result as a DataFrame
res[0].df().head()


,POSITION,WEIGHTS,AS_OF_DATE,REPORTED_MKT_VAL,CURRENCY,LATEST_CHANGE,ORIG_IDS,ID()
ID,,,,,,,,
AAPL US Equity,171835854.0,6.135604,2020-11-24,1.979034e+10,USD,811020.0,SPY US Equity,AAPL US Equity
MSFT US Equity,80888141.0,5.363133,2020-11-24,1.729874e+10,USD,381780.0,SPY US Equity,MSFT US Equity
AMZN US Equity,4550976.0,4.399397,2020-11-24,1.419022e+10,USD,21504.0,SPY US Equity,AMZN US Equity
FB US Equity,25698610.0,2.206318,2020-11-24,7.116459e+09,USD,121296.0,SPY US Equity,FB US Equity
GOOGL US Equity,3211411.0,1.756200,2020-11-24,5.664608e+09,USD,15120.0,SPY US Equity,GOOGL US Equity


For more information about Fund BQL Queries, see <a href="../exampleroot/7b43c540fe28465695d73c1b267a111a/0 Welcome.ipynb">Example Projects &rarr; BQL Resources &rarr; BQL for Funds</a>

[&uarr; Return to Top](#bql-asset-top)

---
<a id="bqlac-econ"></a>
<h1>BQL for Economics<br><span style="color: orange;">Special Functions & Universes for Economics</span></h1>

Bloomberg sources economic data from reliable agencies such as National Statistics Bureaus and other government agencies. However, finding and manipulating economic data on Bloomberg can be a challenge. Users may be familiar with functions such as ECST and ECWB. BQL makes this task a lot easier by simply requesting a country/region, standardizing economic concepts and specifying the transformations needed within the parameters. 


The example below will pull changes in GDP growth for the United States since 2010.
- Specify the country with the country code and <code>' Country'</code>
- <code>period_offset</code> denotes the relative period where 0A is the most recent period and -1A the year before that. You can see that we're extending into two future forecasted periods
- <code>period_type</code> denotes the periodicity of the data, such as 'M','Q','A'

In [12]:
# Run this cell with the Run button in the Jupyter toolbar
# Define the universe
cntry = 'US Country'
# Define the data items - BQL aligns historical and forecasted annual GDP
gdp = bq.data.gdp(period_offset = bq.func.range('-10A','2A'), pt ='a')
# Request data, handle response
req = bql.Request(cntry, {'GDP':gdp})
res = bq.execute(req)
data0 = res[0].df()
data0

,PERIOD_REFERENCE_DATE,TRANSFORM,REPORTED_DATE,CONCEPT,PERIOD_OVER_PERIOD,ECO_SOURCE,PERIOD,PERIOD_END_DATE,SEASONALITY,GDP
ID,,,,,,,,,,
US Country,2009-12-31,PCT,NaT,GDP,YOY,Bloomberg,2009A,2009-12-31,,-2.8
US Country,2010-12-31,PCT,NaT,GDP,YOY,Bloomberg,2010A,2010-12-31,,2.6
US Country,2011-12-31,PCT,NaT,GDP,YOY,Bloomberg,2011A,2011-12-31,,1.6
US Country,2012-12-31,PCT,NaT,GDP,YOY,Bloomberg,2012A,2012-12-31,,2.2
US Country,2013-12-31,PCT,NaT,GDP,YOY,Bloomberg,2013A,2013-12-31,,1.8
US Country,2014-12-31,PCT,NaT,GDP,YOY,Bloomberg,2014A,2014-12-31,,2.5
US Country,2015-12-31,PCT,NaT,GDP,YOY,Bloomberg,2015A,2015-12-31,,3.1
US Country,2016-12-31,PCT,NaT,GDP,YOY,Bloomberg,2016A,2016-12-31,,1.7
US Country,2017-12-31,PCT,NaT,GDP,YOY,Bloomberg,2017A,2017-12-31,,2.3


For more information about Economics BQL Queries, see <a href="../exampleroot/b530936cf22a4f4b82b80b80b8932ba0/0 Welcome.ipynb">Example Projects &rarr; BQL Resources &rarr; BQL for Economics</a>

[&uarr; Return to Top](#bql-asset-top)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="../2. Launch/4 Launch Project.ipynb">&larr; Back to Launch: Final Project</a>&emsp;&emsp;
    <a href="#bql-asset-top">&uarr; Return to Top</a>&emsp;&emsp;
    <a href="2 Dashboards.ipynb">Continue to Orbit: Dashboards &rarr;</a>
    <br>
    <br>
    <a href="../Welcome.ipynb#welcome-top" style="font-size: 12px;">Return to the Welcome Page</a>
</p>